In [1]:
import pandas as pd
from lib import search_display

/Users/serickson-local/src/ucsb-writing-data/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# import reflections data
df = pd.read_csv("data/reflections.csv")
# (optional) only search on a subset of the data
# df = df[ df.DEPT0 == "EEMB/MCDB"]

In [9]:
# search using OpenAI and display 10 results
search_display("rhetorical analysis", df, limit=10, model="openai")

(0.7872) Rhetorical analysis had us looking at topics such as analyzing what we read having us go into a deeper understanding as to why an author uses specific language in order to demonstrate and emphasize different aspects of the story. It also had me looking and understanding the genre in order to better understand why said emphasize was used in certain parts and looking at things such as word choice that best fits and makes up the genre.

(0.7903) The Rhetorical Analysis is something I'm very comfortable with. In school we used political speeches, or even literature pieces, poems, etc. to analyze the speaker's use of rhetorical devices, language, tone, and theme. These are all key aspects in determining and evaluating how and why the speaker intends to reach his/her goal. I know the assignment would entitle a substantial use of commentary along with identifying and analyzing the author's use of rhetoric. The Genre Translation assignment however, was much more confusing. I have some experience with research papers and overall synthesis writing but it's not as evident as the Rhetorical Analysis assignment. 

(0.8169) The "Rhetorical Analysis" assignment was similar to one I have completed in the past in my English 10 class. My English teacher, Mrs. Greenberg, at the time had us taking notes on the three central appeals that are discussed in rhetoric: logos, ethos, and pathos. She instructed us to focus on examples of the rhetorical triangle, commercials, tv shows, songs, etc. By analyzing these, trained me to look at texts, arguments and speeches in terms of how they are designed to persuade the audience.

(0.8241) I have had plenty of experience with completing assignments based on analyzing other texts and the influence rhetorical choices have on those texts. I have written many Rhetorical Analysis essays in which I am tasked with reading a text (whether it be a letter, a speech, an autobiography, etc.) and analyzing what the author's perspective is based on the rhetorical choices they made. This experience relates to these assignments as in both cases the task is to analyze and form your own opinions on a given topic. Whether it be analyzing the genre of a text or forming your own opinions based on an argument created by the author, you must analyze the effect that many different factors have on he outcome of a writing piece.

(0.8246) I am very familiar with these assignments, especially the rhetorical analysis. In the past, I have written many rhetorical analysis and grew very comfortable writing them. This type of essay looks at a text in terms of rhetoric, meaning it is less concerned with what the author is saying than with how they say it: their goals, techniques, and appeals to the audience.A rhetorical analysis is structured similarly to other essays: an introduction presenting the thesis, a body analyzing the text directly, and a conclusion to wrap it up. Three central appeals discussed in rhetoric, established by the philosopher Aristotle are logos, ethos, and pathos.  These three appeals are all treated as integral parts of rhetoric, and a given author may combine all three of them to convince their audience.

(0.8380) I can only speak toward rhetorical analysis, as that is the only assignment style I remember doing. I have analyzed speeches like those of Kennedy's Rice Speech, as well as short stories of fiction, and assorted texts in High School. However I always did somewhat poorly, likely due to a lack of a clear understanding of how to spot rhetorical strategies.

(0.8415) The "Rhetorical Analysis" assignment has a resemblance to an essay I had to write in my junior year of high school. In the assignment, it goes into detail about the use of analyzing texts and even political speeches, "Analyzing political speeches can help you discern what the speaker is trying to accomplish as well as how and why." This reminded me of the essay I wrote because my teacher had me analyze Joe Ehrmann's "Be A Man" TED talk and break down the rhetorical devices he had used in his speech. To accomplish this, I focused on the Ethos, Logos, and Pathos that he utilized and explained why it made his speech more engaging towards the audience. Both assignments link the rhetorical contexts to how it influences the text and or delivery.

(0.8456) The Rhetorical Analysis project is very similar to some of the assignments I did before in my writing class. To be specific, I have analyzed a famous political speech called I Have a Dream. The teacher asked me to analyze these rhetorical devices, like how the repetition and metaphors were used in the speech, and how the speaker used devices to pursue his goal of the speech. I also analyzed rhetorically other political essays or speeches during my writing courses, so such experiences helped me be prepared for future writing assignments and projects.

(0.8458) In my high school writing courses we heavily practiced rhetorical analysis. One memorable instance was when my class did a rhetorical analysis of Narrative of the Life of Fredrick Douglass. That was the first time I was introduced to rhetorical analysis. I was instructed to select a passage and to write a brief paragraph about the author's message, intended audience, context, and rhetoric used. This was all to be achieved with clarity through a strong claim and supporting evidence. Many of my following assignments relating to rhetorical analysis developed my ability to analyze passages as well as constructing my own reflections. 

(0.8464) Rhetorical analysis has been the key to success in my English classes, and in my general personal writing. Analyzing various texts, whether they are nonfiction, fiction, novels, or even scientific journals can improve one's writing skills greatly. Thus, one is able to write in a more exciting, interesting, and sophisticated manner. Through analyzing rhetoric and understanding the difference between genres, I have learned to choose the appropriate tone, style, and diction for different audiences. When I reflect on my writing in school, I realize that my rhetorical tone changes greatly when I am writing about my scientific findings in environmental studies, versus when I am creatively writing a poem in English class. Both the above assignments challenge students to think deeply about rhetoric, and how rhetoric applies through different genres. 

In [10]:
search_display("rhetorical analysis", df, limit=10, model="mxbai")

(9.6800) In my high school writing courses we heavily practiced rhetorical analysis. One memorable instance was when my class did a rhetorical analysis of Narrative of the Life of Fredrick Douglass. That was the first time I was introduced to rhetorical analysis. I was instructed to select a passage and to write a brief paragraph about the author's message, intended audience, context, and rhetoric used. This was all to be achieved with clarity through a strong claim and supporting evidence. Many of my following assignments relating to rhetorical analysis developed my ability to analyze passages as well as constructing my own reflections. 

(9.9868) The assignment Rhetorical Analysis asks to "analyze how the rhetorical context influences the way the text is written or designed." In my junior year of high school, my English teacher would have us read DBQs of old presidential speeches during times of hardship or watch more recent famous speeches such as Greta Thunberg use her voice to bring a sense of urgency and awareness to global warming. We then had to identify the pathos, logos, and ethos of the texts. In addition, in my AP European History class, we would read written accounts almost everyday, analyzing stating its historical value and deducing cause and effect. These experiences I believe have prepared me for successfully completing assignments like Rhetorical Analysis as I have learned the importance of the role of context in important speeches and texts in my classes. Furthermore, the Response essay wants the student to relate a read world issue to themselves in an argumentative essay. Throughout school I have written many argumentative essays, on

(10.099) The "Rhetorical Analysis" assignment has a resemblance to an essay I had to write in my junior year of high school. In the assignment, it goes into detail about the use of analyzing texts and even political speeches, "Analyzing political speeches can help you discern what the speaker is trying to accomplish as well as how and why." This reminded me of the essay I wrote because my teacher had me analyze Joe Ehrmann's "Be A Man" TED talk and break down the rhetorical devices he had used in his speech. To accomplish this, I focused on the Ethos, Logos, and Pathos that he utilized and explained why it made his speech more engaging towards the audience. Both assignments link the rhetorical contexts to how it influences the text and or delivery.

(10.154) During both the English and German courses leading up to my Abitur I applied  rhetorical analysis in several contexts and genres, including poetry, scientific and technical articles as well as subjective newspaper commentaries and other editorial content. Rhetorical analysis was also a key component in the final exams of my Abitur in both English and German. In my English Abitur I analyzed a critical evaluation by an author who discussed the purpose and history of Wikipedia and also expressed his opinions regarding strengths but also potential problems of the sites usage. The goal of my rhetorical analysis was to reveal potential underlying biases of the author that could have affected his assessment by analyzing the authors choice of words, argumentative devices, and the quality of his citation and source material.

(10.241) I worked on multiple Rhetorical Analysis assignments during my AP English Language class. Our class was to complete a rhetorical analysis of a given prompt - usually a speech, written letter, advertisement, or newspaper article. A specific example that we worked on was an essay that argued why dog adoption was beneficial. The analysis had to include a thesis regarding how effectively the author argued their point and textual evidence to support the thesis. The rhetorical analysis had to be around 500 words and five paragraphs long, with a standard introduction, conclusion, and three main body paragraphs. The only difficulty I had with the assignment was time management, but all in all, I was able to complete the assignment and receive satisfactory credit. Regarding genre translation assignments, we worked on similar assignments in the past, but not to the extent we worked on rhetorical analysis. For example, we were instructed to translate a research article on bees into an argumentative piece on preserving t

(10.269) Some of my previous assignments consisted of writing rhetorical analysis essays on poems and short stories. An example of this would be taking apart and discovering literary devices that the author has used in the short story, "Birthday party" by Katherine Brush, then to furthermore add commentary on the reasons and effects of these devices. My experience with writing rhetorical analysis essays directly relate to the assignment "Rhetorical Analysis", while also giving me the skills needed for the "Memory Project", "Response Essay", and "Genre Translation". As rhetorical analysis requires you to continuously connect back to your thesis and story, I have written many essays on rhetorical analysis that contributes to my preparedness for these assignments. 

(10.333) During my high school IB literature class, I was constantly exposed to multiple genres of literary texts (novels, poems, plays, speeches, etc) and trained to analyse them. On top of understanding the content of a literary text, I learned to identify how and why authors use these rhetorical devices with the "logos, pathos, ethos" methodology. For example, one of my weekly assignment was to watch a TED talk speech and write a short analysis on how the speech emotionally appealed to the audience and effectively conveyed their messages. I have also conducted a project on analysing MLK's famous "I Have A Dream" speech". Therefore, I'm fairly confident with rhetorical analysis.Â 

(10.364) In high school I have done rhetorical analysis, on mostly poems and certain literature books. I cannot remember exactly which ones to be exact, however; for the analysis I talked about the authors diction, attitude, point of view, rhetorical techniques and tone. Although the given assignments aren't asking for us to analyze a poem, I feel that it would be similar because the prompt is asking to analyze the text.  

(10.365) This assignment Rhetorical Analysis is similar to one that I have done in the past, the assignment was about analyzing one of the speeches of President Barack Obama. The purpose of the assignment was to discern what the speaker is trying to accomplish, what type of audience he wants to influence, and highlight rhetorical elements. Write a strong thesis to explain the purpose of the speech, then write 4 paragraphs to create an explanation with a clear central idea, supporting the text with details and examples of the speech. Also, use appropriate language and tone; to end with a clear and simple conclusion. 

(10.373) In past assignments given to me, there is a common theme of analyzing text or media and interpreting them in various ways. A specific type of analysis that I have been assigned is rhetorical analysis which is similar to assignments like these where it is asked to find these rhetorical devices and understand why the authors chose to use these devices and the importance of where they show up in the text. Another similarity that was found between assignments was the use of writing reflections. Reflecting on an event, reading, or any media is an exercise/activity that makes the writer think. 